In [1]:
import utils
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from scipy.sparse import hstack
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
from skrub import TableReport
from jours_feries_france import JoursFeries
from vacances_scolaires_france import SchoolHolidayDates

In [2]:
data = pd.read_parquet("data/train.parquet")
# Sort by date first, so that time based cross-validation would produce correct results
data = data.sort_values(["date", "counter_name"])

data_test = pd.read_parquet("data/final_test.parquet")
# Sort by date first, so that time based cross-validation would produce correct results
data_test = data_test.sort_values(["date", "counter_name"])


In [4]:
external_conditions = pd.read_csv('data/external_data.csv')
external_conditions['date'] = pd.to_datetime(external_conditions['date'])


In [5]:
# Drop columns with more than 40% NaN values
threshold = len(external_conditions) * 0.4
external_conditions = external_conditions.dropna(thresh=threshold, axis=1)

In [ ]:
# Step 1: Sort the `external_conditions` DataFrame by the `date` column
external_conditions = external_conditions.sort_values(by='date')

# Drop columns with more than 40% NaN values
threshold = len(external_conditions) * 0.4
external_conditions = external_conditions.dropna(thresh=threshold, axis=1)

# Step 2: Remove duplicate entries based on the `date` column
external_conditions = external_conditions.drop_duplicates(subset='date')

# Step 3: Convert the 'date' column to datetime
external_conditions['date'] = pd.to_datetime(external_conditions['date'])

# Step 4: Create a complete date range from the minimum to the maximum date in the DataFrame
date_range = pd.date_range(start=external_conditions['date'].min(), end=external_conditions['date'].max(), freq='h')

# Step 5: Create a DataFrame from the date_range
date_range_df = pd.DataFrame(date_range, columns=['date'])

# Step 6: Merge the date_range DataFrame with the external_conditions DataFrame on the 'date' column
full_external_conditions = pd.merge(date_range_df, external_conditions, on='date', how='left')

# Fonction qui fait ce qu'on voulait faire avec ffill et bfill mais a la place prends la valeur la plus proche
def fill_closest_value_all_columns(df):
    """Fill NaN values with the closest value for all numeric columns in the DataFrame."""
    filled_df = df.copy()
    
    for column in filled_df.columns:
        if filled_df[column].dtype.kind in 'biufc':  # Numeric columns
            non_nan_values = filled_df[column].dropna()
            
            def find_closest(value):
                if pd.isna(value):
                    closest_value = non_nan_values.iloc[(non_nan_values - value).abs().argmin()]
                    return closest_value
                return value
            
            filled_df[column] = filled_df[column].apply(find_closest)
    
    return filled_df

# Apply the function to the DataFrame
filled_external_conditions = fill_closest_value_all_columns(full_external_conditions)

/tmp/ipykernel_21689/758736975.py:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start=external_conditions['date'].min(), end=external_conditions['date'].max(), freq='H')
/tmp/ipykernel_21689/758736975.py:34: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  closest_value = non_nan_values.iloc[(non_nan_values - value).abs().argmin()]


In [6]:
# Ensure 'date' column in external_conditions is of datetime type
# filled_external_conditions['date'] = pd.to_datetime(filled_external_conditions['date'])

# Merge the DataFrames
merged_conditions = pd.merge(data, filled_external_conditions, on='date', how='left')

merged_conditions = utils._column_rename(merged_conditions)


merged_conditions_test = pd.merge(data_test, filled_external_conditions, on='date', how='left')

merged_conditions_test = utils._column_rename(merged_conditions_test)


In [7]:
# Ensure "date" is in datetime format
merged_conditions["date"] = pd.to_datetime(merged_conditions["date"], errors="coerce")

# Drop rows with invalid datetime entries
df = merged_conditions.dropna(subset=["date"])

# Extract date and time features
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["weekday"] = df["date"].dt.dayofweek
df["day"] = df["date"].dt.day
df["hour"] = df["date"].dt.hour
df["is_weekend"] = (df["weekday"] >= 5).astype(int)

# Handle school and public holidays
unique_dates = df["date"].dt.date.unique()
d = SchoolHolidayDates()
f = JoursFeries()

try:
    dict_school_holidays = {date: d.is_holiday_for_zone(date, "C") for date in unique_dates}
    df["is_school_holiday"] = df["date"].dt.date.map(dict_school_holidays).fillna(0).astype(int)
except Exception as e:
    print(f"Error with school holidays mapping: {e}")
    df["is_school_holiday"] = 0

try:
    dict_public_holidays = {date: f.is_bank_holiday(date, zone="Métropole") for date in unique_dates}
    df["is_public_holiday"] = df["date"].dt.date.map(dict_public_holidays).fillna(0).astype(int)
except Exception as e:
    print(f"Error with public holidays mapping: {e}")
    df["is_public_holiday"] = 0

In [8]:
# Ensure "date" is in datetime format
merged_conditions_test["date"] = pd.to_datetime(merged_conditions_test["date"], errors="coerce")

# Drop rows with invalid datetime entries
df_test = merged_conditions_test.dropna(subset=["date"])

# Extract date and time features
df_test["year"] = df_test["date"].dt.year
df_test["month"] = df_test["date"].dt.month
df_test["weekday"] = df_test["date"].dt.dayofweek
df_test["day"] = df_test["date"].dt.day
df_test["hour"] = df_test["date"].dt.hour
df_test["is_weekend"] = (df_test["weekday"] >= 5).astype(int)

# Handle school and public holidays
unique_dates = df_test["date"].dt.date.unique()
d = SchoolHolidayDates()
f = JoursFeries()

try:
    dict_school_holidays = {date: d.is_holiday_for_zone(date, "C") for date in unique_dates}
    df_test["is_school_holiday"] = df_test["date"].dt.date.map(dict_school_holidays).fillna(0).astype(int)
except Exception as e:
    print(f"Error with school holidays mapping: {e}")
    df_test["is_school_holiday"] = 0

try:
    dict_public_holidays = {date: f.is_bank_holiday(date, zone="Métropole") for date in unique_dates}
    df_test["is_public_holiday"] = df_test["date"].dt.date.map(dict_public_holidays).fillna(0).astype(int)
except Exception as e:
    print(f"Error with public holidays mapping: {e}")
    df_test["is_public_holiday"] = 0

In [ ]:
TableReport(df)

In [ ]:
TableReport(df_test)

## Test using flaml and the GPU

In [ ]:
# import pandas as pd
# from flaml import AutoML
# from skrub import TableVectorizer
# import logging

# # Set up logging
# logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")
# logger = logging.getLogger()

# # Preprocess the dataset
# X = df.drop(columns=['log_bike_count', 'bike_count'])
# y = df['log_bike_count']

# # Split the data into training and validation sets based on the last 10% of dates
# validation_split_index = int(len(df) * 0.9)
# X_train, X_val = X.iloc[:validation_split_index], X.iloc[validation_split_index:]
# y_train, y_val = y.iloc[:validation_split_index], y.iloc[validation_split_index:]

# # Initialize the TableVectorizer
# vectorizer = TableVectorizer()

# # Fit and transform the training data
# logger.info("Starting data transformation with TableVectorizer.")
# X_train_transformed = vectorizer.fit_transform(X_train)
# X_val_transformed = vectorizer.transform(X_val)
# logger.info("Data transformation completed.")


In [ ]:
# # Initialize AutoML
# automl = AutoML()

# # FLAML settings with GPU support
# automl_settings = {
#     "time_budget": 300,  # Maximum time (in seconds)
#     "task": "regression",  # Regression task
#     "metric": "r2",  # Metric to optimize
#     "estimator_list": ["xgboost", "lgbm", "catboost"],  # Algorithms compatible with GPU
#     "log_file_name": "flaml_gpu.log",  # Logging
#     "seed": 42,  # Reproducibility
#     "custom_hp": {  # Specify static GPU configurations
#         "xgboost": {
#             "tree_method": "gpu_hist",  # Use GPU-accelerated histogram for XGBoost
#         },
#         "lgbm": {
#             "device": "gpu",  # Use GPU for LightGBM
#         },
#         "catboost": {
#             "task_type": "GPU",  # Use GPU for CatBoost
#         },
#     },
#     "verbose": 1,  # Print logs during training
#     "eval_method": "holdout",  # Use validation data explicitly
#     "log_type": "all",
#     "keep_search_state": True,  # Retain search state for detailed logging
#     "fit_kwargs_by_estimator": {  # Avoid passing unsupported arguments
#         "xgboost": {},  # Pass no extra arguments to XGBoost
#         "lgbm": {},     # Pass no extra arguments to LightGBM
#         "catboost": {}, # Pass no extra arguments to CatBoost
#     },
# }

# # Train AutoML
# logger.info("Starting AutoML training.")
# automl.fit(X_train=X_train_transformed, y_train=y_train, X_val=X_val_transformed, y_val=y_val, **automl_settings)
# logger.info("AutoML training completed.")

# # Results
# print("Best estimator:", automl.best_estimator)
# print("Best hyperparameters:", automl.best_config)
# print("Best validation R²:", 1 - automl.best_loss)


## Test using Optuna hyperparameter

In [9]:
from skrub import TableVectorizer
from xgboost import XGBRegressor
import optuna
from sklearn.model_selection import train_test_split

# Preprocess the dataset using TableVectorizer
X = df.drop(columns=['log_bike_count', 'bike_count'])
y = df['log_bike_count']

# # Split the data into training and validation sets based on the last 10% of dates
# validation_split_index = int(len(df) * 0.9)
# X_train, X_val = X.iloc[:validation_split_index], X.iloc[validation_split_index:]
# y_train, y_val = y.iloc[:validation_split_index], y.iloc[validation_split_index:]

# Initialize the TableVectorizer
# vectorizer = TableVectorizer()

# # Fit and transform the training data
# X_train_transformed = vectorizer.fit_transform(X_train)
# X_val_transformed = vectorizer.transform(X_val)


In [ ]:
# # Define the objective function for Optuna
# def objective(trial):
#     param = {
#         'objective': 'reg:squarederror',
#         'n_estimators': trial.suggest_int('n_estimators', 100, 500),
#         'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.2),
#         'max_depth': trial.suggest_int('max_depth', 3, 20),
#         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.9),
#         'random_state': 42,
#         'tree_method': 'gpu_hist',  # Enable GPU support
#         'predictor': 'gpu_predictor'
#     }
#     model = XGBRegressor(**param)
#     model.fit(X_train_transformed, y_train)
#     return model.score(X_val_transformed, y_val)  # Maximizing validation R² score

# # Create a study and optimize the objective function
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=50)

# # Get the best parameters
# best_params = study.best_params

# # Add GPU-specific parameter to the best parameters
# best_params['tree_method'] = 'gpu_hist'  # Ensure GPU is used for the final model

# # Train the final model with the best parameters
# X_transformed = vectorizer.transform(X)  # Transform the entire dataset
# final_model = XGBRegressor(**best_params)
# final_model.fit(X_transformed, y)


In [10]:
import joblib
from xgboost import XGBRegressor
from skrub import TableVectorizer

# Load the best parameters from the pickle file
best_params = joblib.load('xg_boost_best_params.pkl')

# Check if GPU support is available
try:
    import xgboost
    if 'gpu_hist' in best_params.get('tree_method', '') and not xgboost.Booster().attr('gpu_id'):
        print("Warning: XGBoost is not compiled with GPU support. Falling back to CPU.")
        best_params.pop('tree_method', None)  # Remove GPU-specific parameters
        best_params.pop('predictor', None)
except Exception as e:
    print(f"Error while checking GPU support: {e}")

# Initialize the TableVectorizer
vectorizer = TableVectorizer()

# Fit and transform the data
X_transformed = vectorizer.fit_transform(X)

# Train the final model with the best parameters
final_model = XGBRegressor(**best_params)
final_model.fit(X_transformed, y)

# Print model parameters
print("Trained model parameters:")
print(final_model.get_params())


Trained model parameters:
{'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.8189577147756041, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.11986932069472364, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 8, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 374, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': 0.6854480891474511, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}


In [11]:
# Transform the test data using the same vectorizer instance
X_test_transformed = vectorizer.transform(df_test)

# Make predictions
y_pred = final_model.predict(X_test_transformed)

print("Predictions:", y_pred)


Predictions: [0.8988337 0.5563414 1.1762435 ... 3.8181887 2.9900236 2.774225 ]


In [16]:
df_submission = pd.DataFrame(y_pred, columns=["log_bike_count"])
df_submission.index = data_test.index
df_submission.index.name = "Id"
df_submission.to_csv("/Users/felix/Documents/X/Cours Python/Kaggle/submission/test_pipeline.csv", index=True)

In [ ]:
test_data = pd.read_parquet('data/final_test.parquet')
# Merge the DataFrames
merged_conditions = pd.merge(test_data, filled_external_conditions, on='date', how='left')

merged_conditions = utils._column_rename(merged_conditions)

# Ensure "date" is in datetime format
merged_conditions["date"] = pd.to_datetime(merged_conditions["date"], errors="coerce")

# Drop rows with invalid datetime entries
df_test = merged_conditions.dropna(subset=["date"])

# Extract date and time features
df_test["year"] = df_test["date"].dt.year
df_test["month"] = df_test["date"].dt.month
df_test["weekday"] = df_test["date"].dt.dayofweek
df_test["day"] = df_test["date"].dt.day
df_test["hour"] = df_test["date"].dt.hour
df_test["is_weekend"] = (df_test["weekday"] >= 5).astype(int)

# Handle school and public holidays
unique_dates_test = df_test["date"].dt.date.unique()

try:
    dict_school_holidays_test = {date: d.is_holiday_for_zone(date, "C") for date in unique_dates_test}
    df_test["is_school_holiday"] = df_test["date"].dt.date.map(dict_school_holidays_test).fillna(0).astype(int)
except Exception as e:
    print(f"Error with school holidays mapping: {e}")
    df_test["is_school_holiday"] = 0

try:
    dict_public_holidays_test = {date: f.is_bank_holiday(date, zone="Métropole") for date in unique_dates_test}
    df_test["is_public_holiday"] = df_test["date"].dt.date.map(dict_public_holidays_test).fillna(0).astype(int)
except Exception as e:
    print(f"Error with public holidays mapping: {e}")
    df_test["is_public_holiday"] = 0

# Predict using the pipeline
y_pred_test = pipeline.predict(df_test)

In [ ]:
df_submission = pd.DataFrame(y_pred_test, columns=["log_bike_count"])
df_submission.index.name = "Id"
df_submission
df_submission.to_csv("/Users/felix/Documents/X/Cours Python/Kaggle/submission/test_pipeline.csv", index=True)

## En dessous c'est des tests d'avant ca ne fait pas tourner ce qui marche actuellement

In [ ]:
# Add the new category to categorical columns
for col in df.select_dtypes(include=['category']).columns:
	df[col] = df[col].cat.add_categories([0])

# Fill NaN values with 0
df = df.fillna(0)

In [ ]:
y_train = df['log_bike_count'].values
X_train = df.drop(['log_bike_count', "bike_count"], axis=1)

date_cols = ["year", "month", "weekday", "day", "hour", "is_weekend", "is_school_holiday", "is_public_holiday"]
categorical_cols = ["counter_name"]
numerical_cols = [
    'latitude', 'longitude', 'Sea Level Pressure (hPa)', 'Pressure Tendency (hPa/3h)',
    'Pressure Tendency Code', 'Wind Direction (°)', 'Wind Speed (m/s)', 'Air Temperature (°C)',
    'Dew Point Temperature (°C)', 'Relative Humidity (%)', 'Visibility (m)', 'Present Weather Code',
    'Past Weather Code 1', 'Past Weather Code 2', 'Total Cloud Cover (oktas)', 'Cloud Base Height (m)',
    'Lowest Cloud Base Height (m)', 'Low Cloud Type', 'Station Level Pressure (hPa)', '24h Pressure Tendency (hPa)',
    '10min Max Wind Gust (m/s)', 'Max Wind Gust (m/s)', 'Measurement Period Duration', 'Ground State',
    'Snow Height (cm)', 'New Snow Depth (cm)', 'New Snowfall Duration (hours)', 'Rainfall (1h, mm)',
    'Rainfall (3h, mm)', 'Rainfall (6h, mm)', 'Rainfall (12h, mm)', 'Rainfall (24h, mm)',
    'Layer 1 Cloud Cover (oktas)', 'Layer 1 Cloud Type', 'Layer 1 Cloud Base Height (m)'
]


# 1. Apply column transformations
# One-hot encode date columns
date_encoder = OneHotEncoder(handle_unknown="ignore")
date_encoded = date_encoder.fit_transform(X_train[date_cols])

# One-hot encode categorical columns
cat_encoder = OneHotEncoder(handle_unknown="ignore")
cat_encoded = cat_encoder.fit_transform(X_train[categorical_cols])

# Standard scale numerical columns
num_scaler = StandardScaler()
num_scaled = num_scaler.fit_transform(X_train[numerical_cols])

X_transformed = hstack([date_encoded, cat_encoded, num_scaled]).toarray()

# 2. Train the model
model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model.fit(X_transformed, y_train)



In [ ]:
X_test = utils.get_test_data()

In [ ]:
# Merge the DataFrames
merged_conditions = pd.merge(X_test, external_conditions, on='date', how='left')

merged_conditions = utils._column_rename(merged_conditions)
# Ensure "date" is in datetime format
merged_conditions["date"] = pd.to_datetime(merged_conditions["date"], errors="coerce")

# Drop rows with invalid datetime entries
df = merged_conditions.dropna(subset=["date"])

# Extract date and time features
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["weekday"] = df["date"].dt.dayofweek
df["day"] = df["date"].dt.day
df["hour"] = df["date"].dt.hour
df["is_weekend"] = (df["weekday"] >= 5).astype(int)

# Handle school and public holidays
unique_dates = df["date"].dt.date.unique()
d = SchoolHolidayDates()
f = JoursFeries()

try:
    dict_school_holidays = {date: d.is_holiday_for_zone(date, "C") for date in unique_dates}
    df["is_school_holiday"] = df["date"].dt.date.map(dict_school_holidays).fillna(0).astype(int)
except Exception as e:
    print(f"Error with school holidays mapping: {e}")
    df["is_school_holiday"] = 0

try:
    dict_public_holidays = {date: f.is_bank_holiday(date, zone="Métropole") for date in unique_dates}
    df["is_public_holiday"] = df["date"].dt.date.map(dict_public_holidays).fillna(0).astype(int)
except Exception as e:
    print(f"Error with public holidays mapping: {e}")
    df["is_public_holiday"] = 0

In [ ]:
# Process the test data with the same transformations as the training data
# 1. Apply column transformations
# One-hot encode date columns
date_encoded_test = date_encoder.transform(df[date_cols])

# One-hot encode categorical columns
cat_encoded_test = cat_encoder.transform(df[categorical_cols])

# Standard scale numerical columns
num_scaled_test = num_scaler.transform(df[numerical_cols])

# Combine all transformed features
X_test_transformed_numeric = hstack([date_encoded_test, cat_encoded_test, num_scaled_test]).toarray()

# 2. Make predictions
y_pred = model.predict(X_test_transformed_numeric)


In [ ]:
df_submission = pd.DataFrame(y_pred, columns=["log_bike_count"])
df_submission.index.name = "Id"

In [ ]:
df_submission.to_csv("/Users/felix/Documents/X/Cours Python/Kaggle/submission/test_pipeline.csv", index=True)